# 1 Introduction

This notebook purpose is to obtain a clean surveys dataset and to do exploratory data analysis on it (basic statistics and visualizations ).

## 1.1 Imports and Notebook Setup

In [1]:
import descartes
import shapely
import os
import pandas_profiling
import time
import warnings
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import osmnx as ox
import pandas as pd
import mplleaflet as leaflet
from scipy.spatial import KDTree
from shapely.geometry import Point
from matplotlib import style
from matplotlib.ticker import PercentFormatter

In [2]:
warnings.filterwarnings('ignore')

sns.set(style="dark")

%matplotlib inline

# 2 Data pre-procesing

This section contains loading of data, concatenation of similar tables and data type integration.

## 2.1 Load datasets

### 2.1.1 Manually completed surveys

In [3]:
def get_completed_surveys(df):
    '''
    Extracts all rows of a DataFrame that have less NaN proportion
    than the maximun NaN proportion in that DataFrame
    
    Parameters:
    -----------
    df: Pandas DataFrame.
    
    Return:
    -------
    filtered_rows: Pandas DataFrame with filtered rows. 
    '''
    row_na_count = df.isna().sum(axis=1)
    row_na_prop = row_na_count / df.shape[1]
    completed_filters = row_na_prop.unique().max()
    filtered_rows = df[row_na_prop < completed_filters]
    
    return filtered_rows

In [4]:
inputdir = "inputs/manually_collected_surveys/"

complete_surveys_list = []

for file in os.listdir(inputdir):
    survey = pd.read_csv(inputdir+file)
    complete_survey = get_completed_surveys(df=survey)
    complete_surveys_list.append(complete_survey)

complete_surveys = pd.concat(complete_surveys_list, ignore_index=True)
geom = complete_surveys.apply(lambda x: shapely.geometry.Point(x["COORDX"], x["COORDY"]), axis=1)
geo_complete_survey = gpd.GeoDataFrame(complete_surveys, geometry=geom, crs={'init':'epsg:4326'})

In [5]:
# # Project to UTM
# geo_complete_survey_proj = ox.projection.project_gdf(geo_complete_survey, to_crs={'init':'epsg:32718'})

### 2.1.2 Load fulcrum surveys 

In [6]:
# Surveys from SIG 2019-1 
data_dir = 'inputs/fulcrum_surveys/students/'
surveys = []

for file in os.listdir(data_dir):
    surveys.append(pd.read_csv(data_dir+file))

df_surveys = pd.concat(surveys, ignore_index=True)
geosurveys = gpd.GeoDataFrame(
    df_surveys.drop(axis=1, labels='geometry'),
    geometry=df_surveys.apply(
        func=lambda x: shapely.geometry.Point(x['longitude'], x['latitude']),
        axis=1
    ),
    crs = {'init':'epsg:4236'}
)
geosurveys = geosurveys[surveys[0].columns.values.tolist()]

In [7]:
# Surveys from feb-2018
fulcrum_data = pd.read_csv('inputs/fulcrum_surveys/nanostore_survey_food_deserts_id.csv')
fulcrum_geodata = gpd.GeoDataFrame(
    fulcrum_data.drop(axis=1, labels='geometry'),
    geometry=fulcrum_data.apply(
        func=lambda x: shapely.geometry.Point(x['longitude'], x['latitude']),
        axis=1
    ),
    crs = {'init':'epsg:4236'}
)
fulcrum_geodata = fulcrum_geodata[geosurveys.columns]

KeyError: "['changeset_id', 'change_type'] not in index"

In [8]:
# concat surveys
geosurveys = pd.concat([geosurveys, fulcrum_geodata], ignore_index=True)

In [9]:
# # Project to UTM
# geosurveys_proj = ox.projection.project_gdf(geosurveys, to_crs={'init':'epsg:32718'}) 

In [10]:
geosurveys.head()

,abre_fin_de_semana,assigned_to,cadena_frio,cant_carga,cant_merma,cant_sku,change_type,changeset_id,created_at,created_by,...,superficie_bodega,system_created_at,system_updated_at,updated_at,updated_by,vehiculo_recojo,vehiculo_recojo_other,venta_frutas,venta_verduras,version
0,yes,NaN,NaN,NaN,NaN,NaN,create,772f7577-792a-4d6e-a9b6-9183905f7e7c,2019-06-15 16:12:06 UTC,k.aspillagav@alum.up.edu.pe,...,NaN,2019-06-15 16:14:44 UTC,2019-06-15 16:14:44 UTC,2019-06-15 16:14:41 UTC,k.aspillagav@alum.up.edu.pe,NaN,NaN,0,0,1
1,yes,NaN,NaN,NaN,NaN,NaN,update,917f5eef-b851-41c3-961f-400b45aed542,2019-06-15 16:12:06 UTC,k.aspillagav@alum.up.edu.pe,...,NaN,2019-06-15 16:14:44 UTC,2019-06-15 16:15:22 UTC,2019-06-15 16:15:20 UTC,k.aspillagav@alum.up.edu.pe,NaN,NaN,0,0,2
2,yes,NaN,NaN,NaN,NaN,NaN,update,9796d61f-eee5-4f96-bcd8-92a7129c2d60,2019-06-15 16:12:06 UTC,k.aspillagav@alum.up.edu.pe,...,NaN,2019-06-15 16:14:44 UTC,2019-06-15 16:17:08 UTC,2019-06-15 16:17:06 UTC,k.aspillagav@alum.up.edu.pe,NaN,NaN,0,0,3
3,yes,NaN,NaN,NaN,NaN,NaN,create,9034bc4b-79bb-4070-99a9-f1f3b876b3a5,2019-06-15 15:30:41 UTC,k.aspillagav@alum.up.edu.pe,...,NaN,2019-06-15 16:17:28 UTC,2019-06-15 16:17:28 UTC,2019-06-15 16:13:57 UTC,k.aspillagav@alum.up.edu.pe,NaN,NaN,0,0,1
4,yes,NaN,NaN,NaN,NaN,NaN,create,9034bc4b-79bb-4070-99a9-f1f3b876b3a5,2019-06-15 16:14:58 UTC,k.aspillagav@alum.up.edu.pe,...,NaN,2019-06-15 16:17:28 UTC,2019-06-15 16:17:28 UTC,2019-06-15 16:17:11 UTC,k.aspillagav@alum.up.edu.pe,NaN,NaN,0,0,1


In [11]:
geosurveys.shape

(604, 48)

In [12]:
geosurveys.isna().sum()

abre_fin_de_semana             219
assigned_to                    604
cadena_frio                    313
cant_carga                     325
cant_merma                     326
cant_sku                       319
change_type                    512
changeset_id                   512
created_at                       0
created_by                       0
fulcrum_id                       0
geometry                         0
gps_altitude                   288
gps_course                     412
gps_horizontal_accuracy        242
gps_speed                      455
gps_vertical_accuracy          455
hora_apertura_fin_de_semana    367
hora_apertura_semana           218
hora_cierre_fin_de_semana      370
hora_cierre_semana             219
horario_atencion_igual         263
latitude                         0
longitude                        0
lugar_principal                306
motivo_no_oferta               396
motivo_no_oferta_other         528
motivo_si_oferta               395
motivo_si_oferta_oth


## 2.2 Concatenate tables

In [13]:
print(geo_complete_survey.columns, end='\n\n')
print(geosurveys.columns)

Index(['ID', 'NOMBCOMP1', 'DOMICILIO', 'COORDX', 'COORDY', 'in_food_desert',
       'venta_frutas', 'venta_verduras', 'superficie_bodega',
       'num_entregas_general', 'num_recojos_general', 'num_lugares',
       'lugar_principal', 'vehiculo_recojo', 'num_entregas_fv',
       'num_recojos_fv', 'num_viajes_lugar_principal', 'cant_carga',
       'cant_sku', 'cadena_frio', 'cant_merma', 'motivo_no_oferta',
       'geometry'],
      dtype='object')

Index(['abre_fin_de_semana', 'assigned_to', 'cadena_frio', 'cant_carga',
       'cant_merma', 'cant_sku', 'change_type', 'changeset_id', 'created_at',
       'created_by', 'fulcrum_id', 'geometry', 'gps_altitude', 'gps_course',
       'gps_horizontal_accuracy', 'gps_speed', 'gps_vertical_accuracy',
       'hora_apertura_fin_de_semana', 'hora_apertura_semana',
       'hora_cierre_fin_de_semana', 'hora_cierre_semana',
       'horario_atencion_igual', 'latitude', 'longitude', 'lugar_principal',
       'motivo_no_oferta', 'motivo_no_oferta_other'

In [14]:
# concat surveys
all_data = pd.concat([geo_complete_survey, geosurveys], ignore_index=True)

# Complete NaNs in lat-long columns
all_data.latitude = all_data.latitude.fillna(all_data.COORDY)
all_data.longitude = all_data.longitude.fillna(all_data.COORDX)

# Filter relevant variables

selected_vars = ['latitude', 'longitude', 'geometry',
                 'venta_frutas', 'venta_verduras', 'superficie_bodega', 'num_personas',
                 'hora_apertura_semana', 'hora_cierre_semana', 'abre_fin_de_semana',
                 'horario_atencion_igual', 'hora_apertura_fin_de_semana',
                 'hora_cierre_fin_de_semana', 'num_entregas_general',
                 'num_recojos_general', 'num_lugares', 'num_entregas_fv',
                 'num_recojos_fv', 'lugar_principal', 'vehiculo_recojo',
                 'vehiculo_recojo_other', 'num_viajes_lugar_principal', 'cant_carga',
                 'cant_sku', 'cadena_frio', 'cant_merma', 'motivo_si_oferta',
                 'motivo_si_oferta_other', 'motivo_no_oferta', 'motivo_no_oferta_other',
                 'gps_altitude']

all_data = all_data[selected_vars]

In [15]:
all_data.shape

(619, 31)

In [16]:
# View NaN count for each variable
all_data.isna().sum()

latitude                         0
longitude                        0
geometry                         0
venta_frutas                     0
venta_verduras                   0
superficie_bodega               47
num_personas                   212
hora_apertura_semana           233
hora_cierre_semana             234
abre_fin_de_semana             234
horario_atencion_igual         278
hora_apertura_fin_de_semana    382
hora_cierre_fin_de_semana      385
num_entregas_general            37
num_recojos_general             60
num_lugares                    312
num_entregas_fv                326
num_recojos_fv                 325
lugar_principal                312
vehiculo_recojo                340
vehiculo_recojo_other          597
num_viajes_lugar_principal     338
cant_carga                     332
cant_sku                       326
cadena_frio                    318
cant_merma                     333
motivo_si_oferta               410
motivo_si_oferta_other         568
motivo_no_oferta    


## 2.3 Data Cleaning

Verify data types and clean unique values for each column

In [17]:
# Verify data types
all_data.dtypes

latitude                        float64
longitude                       float64
geometry                       geometry
venta_frutas                    float64
venta_verduras                  float64
superficie_bodega                object
num_personas                    float64
hora_apertura_semana             object
hora_cierre_semana               object
abre_fin_de_semana               object
horario_atencion_igual           object
hora_apertura_fin_de_semana      object
hora_cierre_fin_de_semana        object
num_entregas_general            float64
num_recojos_general             float64
num_lugares                     float64
num_entregas_fv                 float64
num_recojos_fv                  float64
lugar_principal                  object
vehiculo_recojo                  object
vehiculo_recojo_other            object
num_viajes_lugar_principal      float64
cant_carga                      float64
cant_sku                        float64
cadena_frio                     float64


In [18]:
def superficie_to_km2(x):
    '''Converts string indicating km2 surface to numeric value'''
    if x != None:
        if type(x) == float:
            return x
        elif type(x) == str:
            x_nums = x.split('x')
            return int(x_nums[0])*int(x_nums[1])
    else:
        return pd.np.nan

In [19]:
# Convert superficie_bodega to float
all_data['superficie_bodega'] = all_data['superficie_bodega'].apply(superficie_to_km2)
print(all_data['superficie_bodega'].unique())

[16. 20. 49.  6. 30. 10.  4. 25.  9. nan 45. 64. 32. 28. 12. 18.  8. 48.
 15. 24. 40. 84. 21. 35. 36. 14. 60. 63. 54. 42. 72. 81.  3.  1.  2.  5.
 27. 56. 39.]


In [20]:
print(all_data['cadena_frio'].unique())

[ 1. nan  0.]


In [21]:
print('antes: ', all_data['lugar_principal'].unique())


toreplace_dict = {
    ### Mercado de Santa Anita
    'Mercado de santa anita': 'Gran Mercado Mayorista De Lima',
    'Mercado santa anita': 'Gran Mercado Mayorista De Lima',
    'Mercado de santa Anita': 'Gran Mercado Mayorista De Lima',
    'Santa Anita': 'Gran Mercado Mayorista De Lima',
    'Santa Anita ': 'Gran Mercado Mayorista De Lima',
    'Santa anita': 'Gran Mercado Mayorista De Lima',
    'Mercado de Santa anita': 'Gran Mercado Mayorista De Lima',
    'Mercado Santa Anita ': 'Gran Mercado Mayorista De Lima',
    'Mercado de frutas de santa anita': 'Gran Mercado Mayorista De Lima',
    'Mercado de santa Anita ': 'Gran Mercado Mayorista De Lima',
    'Mercado mayorista de santa anita': 'Gran Mercado Mayorista De Lima',
    'Mayorista Santa Anita ': 'Gran Mercado Mayorista De Lima',
    'Santa anita ': 'Gran Mercado Mayorista De Lima',
    'San Anita': 'Gran Mercado Mayorista De Lima',
    
    ### Mercado Central de Mangomarca
    'Mercado cerca': 'Mercado Central de Mangomarca',
    'Mercado mangomarca': 'Mercado Central de Mangomarca',
    'Mango marca': 'Mercado Central de Mangomarca',
    
    ### Mercado Mayorista Plaza Unicachi Sur
    'Unicachi': 'Mercado Mayorista Plaza Unicachi Sur',
    'Mercado Unicachi': 'Mercado Mayorista Plaza Unicachi Sur',
    'Mercado unicachi': 'Mercado Mayorista Plaza Unicachi Sur',
    'Mercado Unichachi': 'Mercado Mayorista Plaza Unicachi Sur',
    'Mercado de frutas': 'Mercado Mayorista Plaza Unicachi Sur',
    'Mercado unicachi ': 'Mercado Mayorista Plaza Unicachi Sur',
    'Mercado Unicachi ': 'Mercado Mayorista Plaza Unicachi Sur',
    'Unicachi ': 'Mercado Mayorista Plaza Unicachi Sur',
    
    ### Mercado Minorista - La Parada
    'ParadA': 'Mercado Minorista - La Parada',
    'Parada y mercado de frutas': 'Mercado Minorista - La Parada',
    'Parada o mercado': 'Mercado Minorista - La Parada',
    'Parada': 'Mercado Minorista - La Parada',
    'La parada': 'Mercado Minorista - La Parada',
    'La Parada': 'Mercado Minorista - La Parada',
    'Mercado Central - Parada': 'Mercado Minorista - La Parada',
    'Mercado central - parada': 'Mercado Minorista - La Parada',
    'Parada ': 'Mercado Minorista - La Parada',
    'La Prada ': 'Mercado Minorista - La Parada',
    'La parada ': 'Mercado Minorista - La Parada',
    'La Parada ': 'Mercado Minorista - La Parada',
    
    ### Centro Comercial Plaza Villa Sur
    'Villa sur': 'Centro Comercial Plaza Villa Sur',
    'Villa Sur': 'Centro Comercial Plaza Villa Sur',
    
    ### Mercado de Frutas
    'Mercado de Frutas': 'Mercado de Frutas',
    'Mercado mayorista de frutas': 'Mercado de Frutas',
    'Mercado mayorista de fruta': 'Mercado de Frutas',
    'Mercado mayorista de frutas ': 'Mercado de Frutas',
    'Mercado de frutas ': 'Mercado de Frutas',
    'Mercado de frutas y verduras': 'Mercado de Frutas',
    'Mercado de fruta ': 'Mercado de Frutas',
    'Mercado de mayoristas ': 'Mercado de Frutas',
    'Mercado mayorista ': 'Mercado de Frutas',
    'Mercado mayorista': 'Mercado de Frutas',
    'Mercado mayorita': 'Mercado de Frutas',
    '2': 'Mercado de Frutas',
    'Mercado frutas ': 'Mercado de Frutas',
    'Mercado Mayorista ': 'Mercado de Frutas',
    'Mercado de fruta': 'Mercado de Frutas',
    'Mercado Frutas': 'Mercado de Frutas',
    'Mercado mayorista de fruta ': 'Mercado de Frutas',
    'Mercado Mayorista 2': 'Mercado de Frutas',
    'Mercado Mayorista 2 de Frutas ': 'Mercado de Frutas',
    'Mercado de Frutas ': 'Mercado de Frutas',
    'Mercado frutas': 'Mercado de Frutas',
    'Mayorista ': 'Mercado de Frutas',
    'Mercado Mayorista': 'Mercado de Frutas',
    'Mercado frutas ate': 'Mercado de Frutas',
    'Mercado de arriola': 'Mercado de Frutas',
    'Mercado Mayorista Nicolas Arriola': 'Mercado de Frutas',
    'Arriola': 'Mercado de Frutas',
    'Mercado la victoria': 'Mercado de Frutas',
    
    ### Mercado José Carlos Mariategui
    'Mercado mareategui':'Mercado José Carlos Mariategui',
    'Mareategui':'Mercado José Carlos Mariategui',
    'Mercado Mareategui':'Mercado José Carlos Mariategui',
    
    ### Mercado de Surquillo
    'Mercado de Surquillo ': 'Mercado de Surquillo',
    'Mercados de Surquillo': 'Mercado de Surquillo',
    'Mercado Surquillo': 'Mercado de Surquillo',
    
    ### Mercado San Miguel
    'Mercado modelo san miguel': 'Mercado San Miguel',
    'Mercado san miguel': 'Mercado San Miguel',
    
    ### Mercado de Frutas de San Luis
    'Mercado de frutas de san luis': 'Mercado de Frutas de San Luis',
    'Mercado frutas san Luis ': 'Mercado de Frutas de San Luis',
    'Mercado de san luis': 'Mercado de Frutas de San Luis',
    'Mercado San Luis': 'Mercado de Frutas de San Luis',
    
    ### Mercado de Santa Rosa
    'Mercado santa rosa': 'Mercado de Santa Rosa',
    'Mercado santa rosa ': 'Mercado de Santa Rosa',
    'Santa Rosa': 'Mercado de Santa Rosa',
    
    ### Mercado Vencedores
    'Mercado vencedores': 'Mercado Vencedores',
    'Mercado vencedores ': 'Mercado Vencedores',
    
    ### Otros
    'Mercado el bosque': 'Mercado El Bosque',
    'Mercado de productores': 'Mercado Productores',
    'Mercado la laguna': 'Mercado La Laguna',
    'Mercado Modelo': 'Mercado Modelo de Villa el Salvador',
    'Chorrillos': 'Mercado de Chorrillos',
    'San luis': 'Mercado San Luis',
    '28 de julio': 'Mercado Jorge Chavez', # revisado en OSMaps
    'Lurin': 'Mercado Lurin',
    'Comunidad de la sierra ': 'Comunidad de la Sierra',
    'Mercado de frutas (Caquetá) ': 'Mercado Caquetá',
    'Mercado central': 'Mercado Central',
    'Mercado los pasoso': 'Mercado Los Pazos',  # revisado en OSMaps
    
    ### NaNs
    'Mercado': np.nan,
    'Mercado ': np.nan,
    'Mercado municipal': np.nan,
    'El mercado principal': np.nan,
    'Contacto del mercado': np.nan,
    'No sabe quien lo trae': np.nan,
    'Amigo que vende frutas': np.nan,
    'Proveedor fruta': np.nan,
    'Proveedor': np.nan,
    'Ica': np.nan
}

all_data['lugar_principal'] = all_data['lugar_principal'].replace(toreplace_dict)
print('despues: ', all_data['lugar_principal'].unique())

antes:  [nan 'Chorrillos' 'Unicachi' 'Santa Rosa' 'La Parada' 'Mercado municipal'
 'Mercado Central - Parada' 'Mercado de frutas' 'Mercado central - parada'
 'El mercado principal' 'Mercado mayorista' 'Mercado mayorita' 'Mercado'
 'Mercado frutas ate' 'Mercado modelo san miguel' 'Mercado san miguel'
 'Contacto del mercado' 'No sabe quien lo trae' 'Amigo que vende frutas'
 'Proveedor fruta' 'Mercado santa anita' 'Mercado la victoria' 'Mercado '
 'Mercado de fruta' 'Mercado de frutas de santa anita'
 'Mercado de Santa Anita' 'Mercado de Frutas' 'Mercado de productores'
 'Mercado de frutas de san luis' 'Mercado Frutas' 'Arriola'
 'Mercado mayorista de frutas ' 'Mercado de frutas '
 'Mercado de Surquillo ' 'Mercado de Surquillo' 'Mercados de Surquillo'
 'Mercado de frutas y verduras' 'Mercado de frutas (Caquetá) '
 'Mercado de mayoristas ' 'Mercado mayorista ' 'Mercado de arriola'
 'Lurin' 'Mercado de fruta ' 'Santa Anita ' 'Comunidad de la sierra '
 'La parada' 'Ica' 'La Prada ' 'Mayorist

In [22]:
missing_idx = all_data[all_data['motivo_si_oferta'].isna()]['motivo_si_oferta_other'].isna().index

all_data.loc[missing_idx, 'motivo_si_oferta'] = 'Otros'

In [23]:
print('antes: ', all_data['motivo_si_oferta'].unique())
# toreplace_dict = {
#     ### Los clientes piden FyVs
#     'Porque los clientes piden frutas': 'Los clientes piden FyVs',
#     'la gente pide para lonchera': 'Los clientes piden FyVs',
#     'es lo que la gente m�s pide': 'Los clientes piden FyVs',
#     'las personas piden para sus hijos': 'Los clientes piden FyVs',
#     'las personas piden': 'Los clientes piden FyVs',
#     'los clientes piden frutas y verduras': 'Los clientes piden FyVs',
#     'la gente pide frutas y verduras': 'Los clientes piden FyVs',
#     'la gente pide y a la vez queda para su consumo': 'Los clientes piden FyVs',
#     ### No hay otros sitios cerca que ofrezcan FyVs
#     'no hay una mercado cerca y la gente quiere comprar de todo': 'No hay otros sitios cerca que ofrezcan FyVs',
#     'No hay un mercado cerca': 'No hay otros sitios cerca que ofrezcan FyVs',
#     ### Es fácil adquirir FyVs
#     ### Otros:
#     'por que tiene la licencia para vender': 'Otros',
#     'Porque hay un colegio cerca': 'Otros',
#     'Por tiempo de colegio': 'Otros',
#     'Porque cree que le puede generar m�s ingresos': 'Otros',
#     'para surtir el negocio': 'Otros',
#     'Por la temporada escolar y para surtir': 'Otros',
#     'Para surtir el negocio': 'Otros',
#     'frutas por temporada': 'Otros',
#     'Para que la bodega sea surtida': 'Otros',
#     'Para surtir el negocio y tener m�s ingresos': 'Otros',
#     'para que exista variedad en los productos': 'Otros',
#     'Por temporada escolar': 'Otros',
#     'surtir el negocio': 'Otros',
#     'Parasurtir negocio':'Otros',
#     'Por la temporada escolar':'Otros'
# }

# all_data_questions['motivo_si_'] = all_data_questions['motivo_si_'].replace(toreplace_dict)
# print('despues: ', all_data_questions['motivo_si_'].unique())

antes:  ['Otros' 'Los clientes piden FyVs' 'Es fácil adquirir FyVs'
 'No hay otros sitios cerca que ofrezcan FyVs']


In [24]:
missing_idx = all_data[all_data['motivo_no_oferta'].isna()]['motivo_no_oferta_other'].isna().index

all_data.loc[missing_idx, 'motivo_no_oferta'] = 'Otros'

In [25]:
print('antes: ', all_data['motivo_no_oferta'].unique())

toreplace_dict = {
  1: 'Las FyVs se malogran',
  '1': 'Las FyVs se malogran',
  '1 y 2': 'Las FyVs se malogran',
  'No demanda': 'Los clientes no piden FyVs'
}

# toreplace_dict = {
#     ### Los clientes no piden FyVs
#     'No demanda': 'Los clientes no piden FyVs',
#     'la gente quiere algo r�pido para comer': 'Los clientes no piden FyVs',
#     ### No tiene los medios para abastecerse
#     'por la movilidad y lo lejos que queda': 'No tiene los medios para abastecerse',
#     'Por la disponibilidad de tiempo': 'No tiene los medios para abastecerse',
#     'Porque no alcanza el tiempo': 'No tiene los medios para abastecerse',
#     'horario': 'No tiene los medios para abastecerse',
#     'No tiene tiempo': 'No tiene los medios para abastecerse',
#     ### Cercanía de lugares con FyVs
#     'Hay un mercado donde venden frutas y verduras': 'Cercania de lugares con FyVs',
#     'Existencia de un mercado dedicado a la fruta': 'Cercania de lugares con FyVs',
#     'tiendas cercanas con fyv': 'Cercania de lugares con FyVs',
#     'Porque hay un mercado al frente': 'Cercania de lugares con FyVs',
#     'Mercado cerca': 'Cercania de lugares con FyVs',
#     'Mercado cercano': 'Cercania de lugares con FyVs',
#     'hay otras bodegas cerca': 'Cercania de lugares con FyVs',
#     'hay un mercado cerca': 'Cercania de lugares con FyVs',
#     'Porque hay un mercado cerca': 'Cercania de lugares con FyVs',
#     'Cercan�a de otros locales que ofrecen FyVs': 'Cercania de lugares con FyVs',
#     ### Las FyVs se malogran
#     '1': 'Las FyVs se malogran',    
#     '1 y 2': 'Las FyVs se malogran',
#     'las frutas se malogran y las verduras no las compran': 'Las FyVs se malogran',
#     'las frutas y verduras se malogran muy r�pido': 'Las FyVs se malogran',
#     'Por el tiempo y se malogran': 'Las FyVs se malogran',
#     'Por el verano se malogran las F y V': 'Las FyVs se malogran',
#     'Se malogran en verano, pero en invierno s� los vende': 'Las FyVs se malogran',
#     'se malogran muy f�cil, no se mantienen fresca': 'Las FyVs se malogran',
#     'En temporada de verano no venden F y V pero en invierno s�': 'Las FyVs se malogran',
#     ### Las FyVs son muy caras
#     'no le sale a cuenta la mercader�a': 'Las FyVs son muy caras',
#     ### Otros
#     'reci�n esta empezando a vender': 'Otros',
#     'no tengo espacio ni conservador': 'Otros',
#     'no tiene espacio': 'Otros',
#     'por estar cerca a la playa': 'Otros',
#     'por el verano no se vende, m�s se vende ropas': 'Otros',
#     'tambi�n vende licores': 'Otros',
#     'Bodega reci�n empezando': 'Otros',
#     'tienda peque�a':'Otros',
#     'trabaja a puerta cerradq':'Otros',
#     'No es de su inter�s vender f y v': 'Otros',
#     'solo le gusta vender abarrotes, nunca ha vendido frutas y verduras':'Otros',
#     'el establecimiento no es adecuado, no hay espacio':'Otros',
#     'no es rubro':'Otros' 
# }


all_data['motivo_no_oferta'] = all_data['motivo_no_oferta'].replace(toreplace_dict)
print('despues: ', all_data['motivo_no_oferta'].unique())

antes:  ['No tiene tiempo' 'Otros' '1' '1 y 2' 'No demanda' 1.0
 'Las FyVs se malogran' 'Los clientes no piden FyVs'
 'No tiene los medios para abastecerse' 'Las FyVs son muy caras'
 'Cercanía de otros locales que ofrecen FyVs']
despues:  ['No tiene tiempo' 'Otros' 'Las FyVs se malogran'
 'Los clientes no piden FyVs' 'No tiene los medios para abastecerse'
 'Las FyVs son muy caras' 'Cercanía de otros locales que ofrecen FyVs']


In [26]:
# all_data_questions['num_entreg'] = all_data_questions['num_entreg'].astype(float)
# all_data_questions['num_recojo'] = all_data_questions['num_recojo'].astype(float)

In [27]:
missing_idx = all_data[all_data['vehiculo_recojo'].isna()]['vehiculo_recojo_other'].isna().index

all_data.loc[missing_idx, 'vehiculo_recojo'] = 'Otros'

In [28]:
print('antes: ', all_data['vehiculo_recojo'].unique())

toreplace_dict = {
    '1': 'Camion articulado',
    '4': 'Carro particular',
    5.0: 'Taxi',
    '6': 'Transporte publico',
    'Si tiene': 'Otros',
    'No tiene': 'Otros'
    }
    
    
# toreplace_dict = {
#     'No tiene': 'Otros',
#     'Si tiene': 'Otros',
#     '1': 'Camion articulado',
#     '4': 'Carro particular',
#     '5': 'Taxi',
#     '6': 'Transporte publico',
#     'moto': 'Moto',
#     'mototaxi': 'Mototaxi',
#     'Transporte Publico': 'Transporte publico',
# }

all_data['vehiculo_recojo'] = all_data['vehiculo_recojo'].replace(toreplace_dict)
print('despues: ', all_data['vehiculo_recojo'].unique())

antes:  ['No tiene' 'Si tiene' 'Otros' '4' '6' '1' 5.0 'Carretilla' 'Taxi'
 'Carro particular' 'Transporte publico' 'Van' 'Camion simple']
despues:  ['Otros' 'Carro particular' 'Transporte publico' 'Camion articulado'
 'Taxi' 'Carretilla' 'Van' 'Camion simple']


In [29]:
print('antes: ', all_data['venta_frutas'].unique())
# all_data_questions['venta_frut'] = all_data_questions['venta_frut'].replace(to_replace=['0','1',0,1], value=['No','Si','No','Si'])
# print('despues: ', all_data_questions['venta_frut'].unique())

antes:  [0. 1.]


In [30]:
print('antes: ', all_data['venta_verduras'].unique())
# all_data_questions['venta_verd'] = all_data_questions['venta_verd'].replace(to_replace=['0','1',0,1], value=['No','Si','No','Si'])
# print('despues: ', all_data_questions['venta_verd'].unique())

antes:  [0. 1.]


In [31]:
all_data.to_file('outputs/clean_surveys.geojson', driver='GeoJSON')

## 2.4 Create new variables

In [32]:
def set_ventaFyV(x):
    '''Return category acoording to survey defined logic'''
    if (x['venta_frutas'] == 1) & (x['venta_verduras'] == 1): return 'Vende frutas y verduras'
    elif (x['venta_frutas'] == 1) & (x['venta_verduras'] == 0): return 'Vende solo frutas'
    elif (x['venta_frutas'] == 0) & (x['venta_verduras'] == 1): return 'Vende solo verduras'
    else: return 'No vende frutas ni verduras' 

all_data['ventaFyV'] = all_data.apply(set_ventaFyV, axis=1)

all_data['venta_bool'] = all_data['ventaFyV'].replace(
    to_replace = ['Vende solo frutas', 'Vende solo verduras'],
    value = ['Vende frutas y verduras', 'Vende frutas y verduras']
    )

In [33]:
all_data['atencion_semana'] = \
(pd.to_datetime(all_data['hora_cierre_semana'], format='%H:%M') - \
pd.to_datetime(all_data['hora_apertura_semana'], format='%H:%M')).dt.seconds

all_data['atencion_fin_de_semana'] = \
(pd.to_datetime(all_data['hora_cierre_fin_de_semana'], format='%H:%M') - \
pd.to_datetime(all_data['hora_apertura_fin_de_semana'], format='%H:%M')).dt.seconds


all_data['atencion_fin_de_semana'] = all_data.apply(
    lambda row: row['atencion_semana'] if row['horario_atencion_igual'] == 'yes' else row['atencion_fin_de_semana'],
    axis=1
)

all_data['atencion_fin_de_semana'] = all_data.apply(
    lambda row: 0 if row['abre_fin_de_semana'] == 'no' else row['atencion_fin_de_semana'],
    axis=1
)

In [34]:
all_data.isna().sum()

latitude                         0
longitude                        0
geometry                         0
venta_frutas                     0
venta_verduras                   0
superficie_bodega               47
num_personas                   212
hora_apertura_semana           233
hora_cierre_semana             234
abre_fin_de_semana             234
horario_atencion_igual         278
hora_apertura_fin_de_semana    382
hora_cierre_fin_de_semana      385
num_entregas_general            37
num_recojos_general             60
num_lugares                    312
num_entregas_fv                326
num_recojos_fv                 325
lugar_principal                346
vehiculo_recojo                  0
vehiculo_recojo_other          597
num_viajes_lugar_principal     338
cant_carga                     332
cant_sku                       326
cadena_frio                    318
cant_merma                     333
motivo_si_oferta                 0
motivo_si_oferta_other         568
motivo_no_oferta    

In [35]:
all_data.columns

Index(['latitude', 'longitude', 'geometry', 'venta_frutas', 'venta_verduras',
       'superficie_bodega', 'num_personas', 'hora_apertura_semana',
       'hora_cierre_semana', 'abre_fin_de_semana', 'horario_atencion_igual',
       'hora_apertura_fin_de_semana', 'hora_cierre_fin_de_semana',
       'num_entregas_general', 'num_recojos_general', 'num_lugares',
       'num_entregas_fv', 'num_recojos_fv', 'lugar_principal',
       'vehiculo_recojo', 'vehiculo_recojo_other',
       'num_viajes_lugar_principal', 'cant_carga', 'cant_sku', 'cadena_frio',
       'cant_merma', 'motivo_si_oferta', 'motivo_si_oferta_other',
       'motivo_no_oferta', 'motivo_no_oferta_other', 'gps_altitude',
       'ventaFyV', 'venta_bool', 'atencion_semana', 'atencion_fin_de_semana'],
      dtype='object')

In [36]:
# Impute missing values

num_vars_not_missing = ['num_lugares', 'num_entregas_fv', 'num_recojos_fv',
                        'num_viajes_lugar_principal', 'cant_carga',
                        'cant_sku', 'cant_merma']
cat_vars_not_missing = ['lugar_principal', 'vehiculo_recojo',
                        'vehiculo_recojo_other','motivo_si_oferta',
                        'motivo_si_oferta_other', ]

def filter_not_missing(row):
    if row['ventaFyV'] == 'No vende frutas ni verduras':
        for var in all_data.columns:
            if var in num_vars_not_missing:
                row[var] = 0
            if var in cat_vars_not_missing:
                row[var] = 'missing'
    return row

geo = all_data['geometry']
crs = all_data.crs

all_data = all_data.apply(filter_not_missing, axis=1)
all_data = gpd.GeoDataFrame(all_data, geometry=geo, crs=crs)

In [37]:
all_data.isna().sum()

latitude                         0
longitude                        0
geometry                         0
venta_frutas                     0
venta_verduras                   0
superficie_bodega               47
num_personas                   212
hora_apertura_semana           233
hora_cierre_semana             234
abre_fin_de_semana             234
horario_atencion_igual         278
hora_apertura_fin_de_semana    382
hora_cierre_fin_de_semana      385
num_entregas_general            37
num_recojos_general             60
num_lugares                     14
num_entregas_fv                 26
num_recojos_fv                  25
lugar_principal                 47
vehiculo_recojo                  0
vehiculo_recojo_other          296
num_viajes_lugar_principal      38
cant_carga                      32
cant_sku                        26
cadena_frio                    318
cant_merma                      33
motivo_si_oferta                 0
motivo_si_oferta_other         267
motivo_no_oferta    

In [38]:
# Load data and project to UTM
lima_shape = gpd.read_file('inputs/lima_region_metropolitana/lima_metropolitana.shp')
lima_shape_proj = ox.projection.project_gdf(lima_shape, to_crs={'init':'epsg:32718'})

geomkts = gpd.read_file('inputs/geoMarkets')
geomkts.crs = {'init': 'epsg:4326'} # set crs
geomkts_proj = ox.projection.project_gdf(geomkts, to_crs={'init':'epsg:32718'})

# Calculate Food Availability (fa) and Food Desert (fd) zones
fa_zones = geomkts_proj.geometry.buffer(distance=1000, resolution=50) # 1km around markets
lima_geom = lima_shape_proj.geometry.unary_union # lima multipolygon
fa_geom = fa_zones.geometry.unary_union
fd_geom = lima_geom.difference(fa_geom)

In [39]:
print("Food Availbility Zone area (km2):", fa_geom.area/1e+6)

Food Availbility Zone area (km2): 794.3721534965385


In [40]:
print("Lima Metropolitana area (km2):", lima_geom.area/1e+6)

Lima Metropolitana area (km2): 2844.644974462386


In [41]:
print("Food Desert Zone area (km2):", fd_geom.area/1e+6)

Food Desert Zone area (km2): 2069.2077152355687


In [42]:
lima_urban_polygon = gpd.read_file("inputs/lima_urban_area/lima_urban_polygon.geojson")

In [43]:
lima_urban_polygon_proj = ox.projection.project_gdf(lima_urban_polygon, to_crs={'init':'epsg:32718'})

In [44]:
print("Urban Food Availbility Zone area (km2):", lima_urban_polygon_proj.intersection(fa_geom).area/1e+6)

Urban Food Availbility Zone area (km2): 0    788.0374
dtype: float64


In [45]:
print("Lima Metropolitan Urban area (km2):",lima_urban_polygon_proj.area/1e+6)

Lima Metropolitan Urban area (km2): 0    1445.713014
dtype: float64


In [46]:
print("Urban Food Desert Zone area (km2):", lima_urban_polygon_proj.difference(fa_geom).area/1e+6)

Urban Food Desert Zone area (km2): 0    657.675614
dtype: float64


In [47]:
all_data_proj = ox.projection.project_gdf(all_data, to_crs={'init':'epsg:32718'})

In [48]:
start = time.time()
all_data['in_food_desert'] = all_data_proj.geometry.within(fd_geom)
wait_time = time.time() - start
print('points within a polygon calc. time: ', wait_time)

points within a polygon calc. time:  2.409503221511841


In [49]:
all_data.groupby('in_food_desert').size()

in_food_desert
False    479
True     140
dtype: int64

In [50]:
all_data.shape

(619, 36)

In [51]:
# Get district for each survey

lima_shape.distrito = lima_shape.distrito.replace(to_replace='BRE?A', value='BREÑA')

start = time.time()
all_data = gpd.sjoin(all_data, lima_shape[['distrito', 'geometry']].copy(), how='left', op='within')
wait_time = time.time() - start
print('spatial join (point-polygons): ', wait_time)

spatial join (point-polygons):  0.28092241287231445


In [52]:
all_data.distrito.isna().sum()

0

In [53]:
all_data.shape

(619, 38)

In [54]:
all_data.groupby('distrito').size()

distrito
ATE                        37
BARRANCO                   50
CHORRILLOS                  9
LA MOLINA                  13
LINCE                       4
MAGDALENA DEL MAR           3
PUEBLO LIBRE                5
SAN ISIDRO                 46
SAN JUAN DE LURIGANCHO    105
SAN MIGUEL                 84
SANTIAGO DE SURCO          50
SURQUILLO                 101
VILLA EL SALVADOR         112
dtype: int64

In [55]:
# View number of surveys in each district considering food zone
all_data.groupby(['distrito','in_food_desert']).count().iloc[:,0:1]

latitude
distrito               in_food_desert          
ATE                    False                 37
BARRANCO               False                 50
CHORRILLOS             True                   9
LA MOLINA              False                 13
LINCE                  False                  4
MAGDALENA DEL MAR      False                  2
                       True                   1
PUEBLO LIBRE           False                  5
SAN ISIDRO             False                 18
                       True                  28
SAN JUAN DE LURIGANCHO False                 67
                       True                  38
SAN MIGUEL             False                 84
SANTIAGO DE SURCO      False                 37
                       True                  13
SURQUILLO              False                101
VILLA EL SALVADOR      False                 61
                       True                  51

In [56]:
markets = pd.DataFrame(all_data['lugar_principal'].unique(), columns=['lugar_principal'])

In [57]:
def geocode(address, city, country):
    '''
    Extends osmnx.geocode to use try/catch for adding city and country to the request.

    Parameters:
    -----------
    address : str
    Requested address to geocode
    city : str
    Expected city where the address is located
    country : str
    Expected city where the address is located

    Returns:
    --------
    geocoding : List of Shapely Geometry Points
    '''
  
    try:
        return ox.geocode(f'{address}, {city}, {country}')
    except:
        try:
            return ox.geocode(f'{address}, {city}')
        except:
            try:
                return ox.geocode(address)
            except:
                return 0, 0 

In [58]:
# Modify market name according to district

markets = markets[markets.lugar_principal != 'Mercado de Santa Rosa']
markets = markets.append(
    pd.DataFrame(
        ['Mercado Central Fevacel','Mercado Santa Rosa de Viña del Mar'],
        columns=['lugar_principal']
    ),
    ignore_index=True
)

all_data[(all_data['distrito'] == 'VILLA EL SALVADOR') &
         (all_data['lugar_principal'] == 'Mercado de Santa Rosa')]['lugar_principal'] = 'Mercado Santa Rosa de Viña del Mar'

all_data[(all_data['distrito'] == 'SAN JUAN DE LURIGANCHO') &
         (all_data['lugar_principal'] == 'Mercado de Santa Rosa')]['lugar_principal'] = 'Mercado Central Fevacel'

In [59]:
# Geocode markets
geom_markets = markets.lugar_principal.apply(geocode, args=('Lima', 'Perú'))
geometries = [shapely.geometry.Point(x, y) for y, x in geom_markets.values]

geomarkets = gpd.GeoDataFrame(
    markets,
    geometry=geometries,
    crs={'init':'epsg:4326'}
)

In [60]:
# Custom point for Mercado La Laguna
geomarkets = geomarkets[geomarkets['lugar_principal']!='Mercado La Laguna']
temp = {
    'lugar_principal': 'Mercado La Laguna',
    'geometry': shapely.geometry.Point(-76.9046382, -12.0384654)
}
temp = pd.DataFrame.from_dict(temp, orient='index').T
geomarkets = geomarkets.append(temp, ignore_index=True)

In [61]:
# Custom point for Mercado Santa Anita
geomarkets = geomarkets[geomarkets['lugar_principal']!='Mercado de Santa Anita']
temp = {
    'lugar_principal': 'Mercado de Santa Anita',
    'geometry': shapely.geometry.Point(-76.957434, -12.043319)
}
temp = pd.DataFrame.from_dict(temp, orient='index').T
geomarkets = geomarkets.append(temp, ignore_index=True)

In [62]:
# Comunidad de la Sierra geometry 
geomarkets.loc[geomarkets['lugar_principal']=='Comunidad de la Sierra','geometry'] = np.nan # Not found

In [63]:
geomarkets = geomarkets.dropna()

In [64]:
drop_index = geomarkets[geomarkets['lugar_principal']=='missing'].index
geomarkets.drop(labels=drop_index, inplace=True)

In [65]:
# Project to UTM
geomarkets_proj = ox.projection.project_gdf(geomarkets, to_crs={'init':'epsg:32718'})

In [67]:
# Add market geometry and nanostore geometry
nanostore_market_dist = pd.merge(
    left=all_data[['lugar_principal','geometry']].to_crs({'init':'epsg:32718'}),
    right=geomarkets_proj,
    how='left',
    on='lugar_principal',
    suffixes=('','_markets')
)

In [73]:
geomarkets

,lugar_principal,geometry
1,Mercado de Chorrillos,POINT (-77.02238 -12.16909)
2,Mercado Mayorista Plaza Unicachi Sur,POINT (-76.96467 -12.19929)
3,Mercado Minorista - La Parada,POINT (-77.01113 -12.06425)
5,Mercado de Frutas,POINT (-76.99840 -12.06861)
6,Mercado San Miguel,POINT (-77.09048 -12.08617)
7,Gran Mercado Mayorista De Lima,POINT (-76.94403 -12.03878)
8,Mercado Productores,POINT (-77.05812 -12.00995)
9,Mercado de Frutas de San Luis,POINT (-76.99840 -12.06861)
10,Mercado de Surquillo,POINT (-77.01701 -12.11678)
11,Mercado Caquetá,POINT (-77.04348 -12.03497)


In [ ]:
fig, ax = plt.subplots(1,1)
lima_shape.plot(figsize=(5,10), alpha=0.5, ax=ax)
all_data.plot(color='red', axes=ax, marker='o')
geomarkets.plot(color='green', axes=ax, marker='s')

plt.show()

In [ ]:
fa_geom_ = gpd.GeoDataFrame(geometry=[fa_geom], crs=lima_urban_polygon_proj.crs).to_crs(lima_urban_polygon.crs)

In [ ]:
df_marketsINEI = pd.read_csv("~/Documents/geodude/data/input/database_mercadosINEI.csv")

change_ix = df_marketsINEI[df_marketsINEI['longitude']<-100].index
df_marketsINEI.loc[change_ix,['longitude', 'latitude']] = df_marketsINEI.loc[change_ix,['longitude', 'latitude']]/1000000


gdf_marketsINEI = gpd.GeoDataFrame(data=df_marketsINEI,
                                   geometry=gpd.points_from_xy(df_marketsINEI['longitude'], df_marketsINEI['latitude']),
                                   crs={'init':'epsg:4326'}
                                  )

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,10))

lima_urban_polygon.plot(ax=ax)
gpd.GeoDataFrame(
    geometry=[fa_geom],
    crs=lima_urban_polygon_proj.crs
).to_crs(
    crs=lima_urban_polygon.crs
).plot(color='red', axes=ax, alpha=0.5)
gdf_marketsINEI.plot(ax=ax, markersize=15, marker='*', color="green")
leaflet.show()

In [ ]:
# Euclidean distance from nanostore to market
# nanostore_market_dist.distance(gpd.GeoSeries(nanostore_market_dist['geometry_markets']))

In [ ]:
def manhattan_distance(a, b):
    '''
    Calculates manhattan distance between two Point geometries in UTM

    Parameters:
    -----------
        a: Shapely Point Geometry
        b: Shapely Point Geometry

    Return:
    -------
        d: Float
        Manhattan distance (also known as taxicab or cityblock distance) between
        a and b (if a or b are not Shapely Point Geometries returns np.nan).

    '''
    try:
        assert(type(a)==shapely.geometry.Point)
        assert(type(b)==shapely.geometry.Point)
        d = np.abs(a.x - b.x) + np.abs(a.y - b.y)
    except:
        d = np.nan # Oops!  That was no valid dtype.
    return d

In [ ]:
all_data['distance_to_market'] = nanostore_market_dist.apply(
    lambda x: manhattan_distance(
        x.geometry,
        x.geometry_markets
    ),
    axis=1
) # Manhattan distance from nanostore to market

In [ ]:
nanostore_market_dist

In [ ]:
def get_distance_to_closest_target(orig, target):
    '''
    Calculate distance from many origin points to many target points,
    then filter min distance for each origin point.

    Parameters:
    -----------
        orig: Pandas Series containing shapely Points geometries.
        Origin points
        target: Pandas Series containing shapely Points geometries.
        Target points

    Return:
    -------
        dist_to_closest_target: Pandas Series
        Distance to the closest target point for each orig point
    '''
    distance = pd.DataFrame(data=[], index=orig.index, columns=range(target.shape[0]))
    for i, geom in enumerate(target):
        distance.iloc[:,i] = orig.distance(geom)
    
    dist_to_closest_target = distance.min(axis=1)
    return dist_to_closest_target

In [ ]:
all_data['dist_2closest_mkt'] = get_distance_to_closest_target(all_data_proj.geometry, geomkts_proj.geometry)

In [ ]:
all_data['dist_2closest_mkt'].isna().sum()

In [ ]:
all_data.groupby('in_food_desert').size()

In [ ]:
city_blocks = gpd.read_file('inputs/manzanas_nse/mz_lima_region.shp')

In [ ]:
city_blocks = city_blocks[~city_blocks.NSE07.isna()]

In [ ]:
all_data.columns

In [ ]:
all_data = gpd.sjoin(left_df=all_data,
              right_df=city_blocks[['NSE07','geometry']],
              how='left',
              op='intersects',
              lsuffix='left2',
              rsuffix='right2')

In [ ]:
all_data.isna().sum()

In [ ]:
def fill_nse(x):
    idx = city_blocks.geometry.distance(x['geometry']).idxmin()
    x['NSE07'] = city_blocks.loc[idx, 'NSE07']
    return x

In [ ]:
missing_ix = all_data['NSE07'].isna().index

all_data.loc[missing_ix, :] = all_data.loc[missing_ix, :].apply(fill_nse, axis=1)

In [ ]:
all_data.isna().sum()

In [ ]:
lima_grid = pd.read_csv('inputs/LIM_ClustersData.csv')

lima_grid_point = gpd.GeoDataFrame(
    data=lima_grid,
    crs={'init': 'epsg:4326'},
    geometry=[Point(xy) for xy in zip(lima_grid['lon'], lima_grid['lat'])]
)

# Coordinate system
proj_utm = {'datum': 'WGS84', 'ellps': 'WGS84', 'proj': 'utm', 'zone': 18, 'units': 'm'}
lima_grid_proj = lima_grid_point.to_crs(proj_utm)
buffer = lima_grid_proj.buffer(distance=500)
grid = buffer.envelope
grid = grid.to_crs({'init': 'epsg:4326'})

lima_grid = gpd.GeoDataFrame(
    data=lima_grid,
    crs={'init': 'epsg:4326'},
    geometry=grid
)

In [ ]:
selected_vars_df = lima_grid[['population', 'betweenness_centrality_avg',
                              'kopt+0_Clusters','geometry']]

In [ ]:
all_data = all_data.drop(labels=['index_right','index_right2'], axis=1)

In [ ]:
all_data = gpd.sjoin(left_df=all_data, right_df=selected_vars_df, how='left', op='within')

In [ ]:
all_data['kopt+0_Clusters'] = all_data['kopt+0_Clusters'].astype(str)

In [ ]:
all_data.isna().sum()

In [ ]:
# make search tree with complete vars
geocomplete_ix = all_data[~all_data['index_right'].isna()].index
data = all_data.loc[geocomplete_ix, :]
data_for_search = all_data.loc[geocomplete_ix, ['latitude','longitude']]
search_kdtree = KDTree(data_for_search)

In [ ]:
geomissings_ix = all_data[all_data.index_right.isna()].index
query_data = all_data.loc[geomissings_ix, ['latitude','longitude']]

In [ ]:
d, i = search_kdtree.query(query_data)

In [ ]:
vars_to_geofill = ['population','betweenness_centrality_avg','kopt+0_Clusters']
all_data.loc[geomissings_ix,vars_to_geofill] = data.iloc[i,-3:].values

In [ ]:
all_data['kopt+0_Clusters'] = all_data['kopt+0_Clusters'].astype(str)

In [ ]:
all_data.isna().sum()

In [ ]:
# Save final dataset
all_data.to_file('outputs/newvars_surveys.geojson', driver='GeoJSON')

In [ ]:
sum(all_data[all_data['venta_bool'] == 'Vende frutas y verduras']['num_recojos_fv'] > 0) / all_data[all_data['venta_bool'] == 'Vende frutas y verduras'].shape[0]

In [ ]:
all_data['venta_bool']

In [75]:
geomarkets.to_file("outputs/markets.geojson", driver='GeoJSON')